In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from keras.models import *
from keras.layers import *
from keras.datasets import mnist
import os

In [ ]:
OUT_DIR = './CNN_out'
img_shape = (28,28,1)
epochs = 100000
batch_size = 128
noise = 100
sample_interval = 100

In [ ]:
(x_train, _), (_,_) = mnist.load_data()
print(x_train.shape)

x_train = x_train/127.5 - 1
x_train = np.expand_dims(x_train, axis=3)
print(x_train.shape)

(60000, 28, 28)
(60000, 28, 28, 1)


In [ ]:
generator = Sequential()
generator.add(Dense(256*7*7, input_dim=noise))
generator.add(Reshape((7, 7, 256)))
generator.add(Conv2DTranspose(128,kernel_size=3,strides=2, padding='same')) # conv 후 maxpool
generator.add(BatchNormalization()) #값이 커져서 발산하지 않도록 배치별로 한번씩 스켈링 해줌 (최소 최대 정규화)
generator.add(LeakyReLU(alpha=0.01))
generator.add(Conv2DTranspose(64,kernel_size=3,strides=1, padding='same'))
generator.add(BatchNormalization())
generator.add(LeakyReLU(alpha=0.01))
generator.add(Conv2DTranspose(1,kernel_size=3,strides=2, padding='same'))
generator.add(Activation('tanh'))
generator.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 12544)             1266944   
                                                                 
 reshape (Reshape)           (None, 7, 7, 256)         0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 14, 14, 128)      295040    
 nspose)                                                         
                                                                 
 batch_normalization (BatchN  (None, 14, 14, 128)      512       
 ormalization)                                                   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 14, 14, 128)       0         
                                                                 
 conv2d_transpose_1 (Conv2DT  (None, 14, 14, 64)       7

In [ ]:
discriminator = Sequential()
discriminator.add(Conv2D(32, kernel_size=3, strides= 2, padding='same', input_shape= img_shape)) #strides = 2 는 두칸 건너뛰기
discriminator.add(LeakyReLU(alpha=0.01))
discriminator.add(Conv2D(64, kernel_size=3, strides= 2, padding='same'))
discriminator.add(LeakyReLU(alpha=0.01))
discriminator.add(Conv2D(128, kernel_size=3, strides= 2, padding='same'))
discriminator.add(LeakyReLU(alpha=0.01))
discriminator.add(Flatten())
discriminator.add(Dense(1, activation='sigmoid'))
discriminator.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 14, 14, 32)        320       
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 14, 14, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 7, 64)          18496     
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 7, 7, 64)          0         
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 128)         73856     
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 4, 4, 128)         0         
                                                                 
 flatten (Flatten)           (None, 2048)             

In [ ]:
discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
discriminator.trainable = False #discriminater 학습 안되게 막아줌


gan_model = Sequential()
gan_model.add(generator)
gan_model.add(discriminator)
gan_model.summary()
gan_model.compile(loss='binary_crossentropy', optimizer='adam')

#이진분류기 타겟
real = np.ones((batch_size,1)) #1로 채워진 행렬을 만들어줌/ 개수는 배치사이즈만큼
fake = np.zeros((batch_size,1))

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 28, 28, 1)         1637121   
                                                                 
 sequential_1 (Sequential)   (None, 1)                 94721     
                                                                 
Total params: 1,731,842
Trainable params: 1,636,737
Non-trainable params: 95,105
_________________________________________________________________


In [ ]:
for epoch in range(epochs):
    idx = np.random.randint(0,x_train.shape[0],batch_size) #0~6사이의 인트값을 랜덤하게 뽑는다
    real_imgs = x_train[idx]

    z = np.random.normal(0,1,(batch_size, noise))
    fake_imgs = generator.predict(z)

    d_hist_real = discriminator.train_on_batch(real_imgs, real) #이것만 학습하고 끝남(1 epoch)
    d_hist_fake = discriminator.train_on_batch(fake_imgs, fake)

    d_loss, d_acc = np.add(d_hist_fake, d_hist_real)*0.5 #평균값


    if epoch %2 == 0:
        z = np.random.normal(0,1,(batch_size, noise))
        gan_hist = gan_model.train_on_batch(z, real) #1이라고 답하게 학습 (generator만 학습됨)

    if epoch % sample_interval == 0: #100에폭마다 한번씩
        print('%d, [D loss: %f, acc.: %.2f%%], [G loss: %f]'
              %(epoch, d_loss, d_loss, gan_hist))
        row = col = 4
        z = np.random.normal(0,1,(row*col, noise))
        fake_imgs = generator.predict(z)
        fake_imgs = 0.5*fake_imgs + 0.5
        _, axs = plt.subplots(row, col, figsize=(5,5),sharey=True, sharex=True)
        cont = 0
        for i in range(row):
            for j in range(col):
                axs[i,j].imshow(fake_imgs[cont, :, :, 0], cmap='gray')
                axs[i,j].axis('off')
                cont += 1
        path = os.path.join(OUT_DIR, 'img-{}'.format(epoch+1))
        plt.savefig(path)
        plt.close()

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
4/4 [==============================] - 0s 7ms/step
13000, [D loss: 0.282266, acc.: 0.28%], [G loss: 2.932652]
4/4 [==============================] - 0s 4ms/step
13100, [D loss: 0.492945, acc.: 0.49%], [G loss: 2.851820]
4/4 [==============================] - 0s 6ms/step
13200, [D loss: 0.424570, acc.: 0.42%], [G loss: 3.267383]
4/4 [==============================] - 0s 8ms/step
13300, [D loss: 0.278021, acc.: 0.28%], [G loss: 3.291396]
4/4 [==============================] - 0s 4ms/step
13400, [D loss: 0.271009, acc.: 0.27%], [G loss: 3.444918]
4/4 [==============================] - 0s 7ms/step
13500, [D loss: 0.304017, acc.: 0.30%], [G loss: 3.226607]
4/4 [==============================] - 0s 7ms/step
13600, [D loss: 0.187027, acc.: 0.19%], [G loss: 3.504115]
4/4 [==============================] - 0s 6ms/step
13700, [D loss: 0.267834, acc.: 0.27%], [G loss: 2.630406]
4/4 [==============================] - 0s 6ms/step
13800, [D loss: 0.219327, acc.: 0